
# Ticket Data

## Workflow Stages
__1. Read the dataset.__ 

__2. new column with hostname__

__3. fields with datetime should be of type datetime.__


### Importing the libraries

In [1]:
# Data Visualization & Wrangling
import pandas
import pysnow

# For changing the string type to datetime type
import datetime

### __1. Reading the dataset.__ 

In [2]:
#Reading the csv file
data = pandas.read_csv("incident.csv")

### Getting the information about the data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
number               19 non-null object
caller_id            19 non-null object
description          19 non-null object
short_description    19 non-null object
sys_created_on       19 non-null object
cmdb_ci              18 non-null object
priority             19 non-null object
assignment_group     19 non-null object
severity             19 non-null object
dtypes: object(9)
memory usage: 1.4+ KB


In [4]:
data.head()
#We need to work with the column 'description' and 'sys_created_on'

,number,caller_id,description,short_description,sys_created_on,cmdb_ci,priority,assignment_group,severity
0,INC6110268,IBM NetCool,Summary: Oracle avh1002p Auto-Extend tablespac...,dbr32xjbx_A Oracle avh1002p Auto-Extend tables...,2020-02-03 12:29:29,dbr32xjbx,2 - Medium,E_XXX_XXX_DIST-DAT-DATABASE_IBM,3 - Low
1,INC6168056,IBM NetCool,ORGTICKETGROUP=cd4e614d0fd49600a8a0e388b1050eb...,dbq21gbfx High space used (75%. 25856 MB free)...,2020-02-17 04:19:07,DBQ21GBFX,3 - Low,E_XXX_XXX_DIST-DAT-DATABASE_IBM,3 - Low
2,INC6170398,IBM NetCool,ORGTICKETGROUP=494e614d0fd49600a8a0e388b1050eb...,dcp00cnhw Windows Service:ShellHWDetection has...,2020-02-17 13:23:39,DCP00CNHW,3 - Low,N_XXX_XXX_Dynamic_Automation_Support,3 - Low
3,INC6169794,IBM NetCool,Summary: CRITICAL JOB P01C35D HAS NOT RUN ON H...,A01A CRITICAL JOB P01C35D HAS NOT RUN ON HPDC,2020-02-17 11:46:29,AF Operator-N-P,2 - Medium,N_USA_XXX_CENTRAL ORDER ENTRY,3 - Low
4,INC6167105,IBM NetCool,Summary: Cleared and Re-fired: all_avgresp_gyn...,apl15xjbx Cleared and Re-fired: all_avgresp_gy...,2020-02-16 13:13:19,apl15xjbx,3 - Low,E_XXX_XXX_DIST-AHS-WEBMSC_IBM,3 - Low


### __2. Creating new column 'hostname' from description field__

In [5]:
# Extracting all the resource ids from the description of incidents
data['HOSTNAME']=data['description'].str.extract('(ResourceId:\s\w+)')
data['HOSTNAME']=data['HOSTNAME'].str.replace('ResourceId:\s', '')

In [6]:
data.head()
#New column 'Hosts' has been added 

,number,caller_id,description,short_description,sys_created_on,cmdb_ci,priority,assignment_group,severity,HOSTNAME
0,INC6110268,IBM NetCool,Summary: Oracle avh1002p Auto-Extend tablespac...,dbr32xjbx_A Oracle avh1002p Auto-Extend tables...,2020-02-03 12:29:29,dbr32xjbx,2 - Medium,E_XXX_XXX_DIST-DAT-DATABASE_IBM,3 - Low,dbr32xjbx_A
1,INC6168056,IBM NetCool,ORGTICKETGROUP=cd4e614d0fd49600a8a0e388b1050eb...,dbq21gbfx High space used (75%. 25856 MB free)...,2020-02-17 04:19:07,DBQ21GBFX,3 - Low,E_XXX_XXX_DIST-DAT-DATABASE_IBM,3 - Low,dbq21gbfx
2,INC6170398,IBM NetCool,ORGTICKETGROUP=494e614d0fd49600a8a0e388b1050eb...,dcp00cnhw Windows Service:ShellHWDetection has...,2020-02-17 13:23:39,DCP00CNHW,3 - Low,N_XXX_XXX_Dynamic_Automation_Support,3 - Low,dcp00cnhw
3,INC6169794,IBM NetCool,Summary: CRITICAL JOB P01C35D HAS NOT RUN ON H...,A01A CRITICAL JOB P01C35D HAS NOT RUN ON HPDC,2020-02-17 11:46:29,AF Operator-N-P,2 - Medium,N_USA_XXX_CENTRAL ORDER ENTRY,3 - Low,NaN
4,INC6167105,IBM NetCool,Summary: Cleared and Re-fired: all_avgresp_gyn...,apl15xjbx Cleared and Re-fired: all_avgresp_gy...,2020-02-16 13:13:19,apl15xjbx,3 - Low,E_XXX_XXX_DIST-AHS-WEBMSC_IBM,3 - Low,apl15xjbx


### Accessing the Database and fetching the server type from ServiceNow developer website

In [7]:
import cred
class_name_list=[]
c = pysnow.Client(instance='dev76764', user=cred.user, password=cred.password) #Creting the connection

# for item in list(data['HOSTNAME']):
#     response = c.resource(api_path='/table/cmdb_ci').get(query={'name': item}) #Accessing the database table #u'name': u'dcp00cnhw'
    
    
#     #Handling the exception
#     try :
#         class_name = response["sys_class_name"][8:].split("_") #u'sys_class_name': u'cmdb_ci_win_server'
#         class_name = " ".join(class_name)
#         class_name = class_name.encode("UTF-8").upper()
#         class_name_list.append(class_name)
        
#     except Exception :
        
#         class_name_list.append("No Record")
        
# item='dcp00cnhw'
# response = c.resource(api_path='/table/cmdb_ci').get(query={'name': item}, stream=True)
# response.one()

# response["sys_class_name"][8:]



response = c.resource(api_path='/table/cmdb_ci').get()
data_1 = pandas.DataFrame.from_dict(response.all())
datanew=data_1[["name","sys_class_name"]]
datanew.head()
data.head()
datanew.rename(columns = {'name':'HOSTNAME',"sys_class_name":"Class"}, inplace = True) 
datanew.head()
data_merged = pandas.merge(data, datanew, on='HOSTNAME', how='inner')
data_merged.head(20)


ValueError: No JSON object could be decoded

In [ ]:
# data["Class"]=pandas.DataFrame(class_name_list)

In [ ]:
data.head(10)

### __3. Converting datetime fields from string type to datetime type.__

In [ ]:
#Converting 'sys_created_on' from string type to datetime type
data["sys_created_on"] = pandas.to_datetime(data["sys_created_on"])

#### Here we need to change the 'sys_created_on' type from string type to datetime type   

In [ ]:
data.info()
#Here 'sys_created_on' changed to datetime type

### __4. Accessing the mariadb.__

In [10]:
import pymysql.cursors
connection = pymysql.connect(host='localhost',
                             user='parsa',
                             password='Welcome123@',
                             db='my_db',
                             charset='tf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


with connection.cursor() as cursor:

    cursor.execute("""
                        insert into students(student_id,student_name,admission_date)
                        values (%s,'%s',%s)
                        """%(2, 'Andrea',2020-01-01))
    
connection.commit()

with connection.cursor() as cursor:
    
    cursor.execute("""
                        select * from students;
                            """
                            )
    result=cursor.fetchall()

result

AttributeError: 'NoneType' object has no attribute 'encoding'

In [9]:
table_df = pandas.DataFrame.from_dict(result)
table_df

NameError: name 'result' is not defined